In [14]:
""" Note I altered tmin to 2.3, tmax to 4.3, and baseline to (None, 3) before running this notebook """

' Note I altered tmin to 2.3, tmax to 4.3, and baseline to (None, 3) before running this notebook '

CONSTANTS

In [1]:
from helpers import extract_eeg, preprocess, prepare_data, run_model, evaluate_model, compute_empirical_chance
from pathlib import Path
import os
import matplotlib.pyplot as plt

In [2]:
MODEL = 'lda'                       # classifier ('svm', 'rf', 'lda', 'xgb')
PCA, N_COMPONENTS = True, 20        # PCA downsampling
TIME_DOWNSAMPLE_FACTOR = 4          # Time downsampling factor
N_FOLDS = 4                         # num. folds for K_folds

In [ ]:
import numpy as np
import mne

channel = 'Cz'
window_size = 50  # Num samples in moving average window

def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size) / window_size, mode='valid')

def compute_grand_average_with_error(epochs, channel, window_size):
    data = epochs.get_data(picks=channel)  # Shape: (n_epochs, n_times)
    smoothed_signals = []

    for epoch_signal in data:  # Each epoch_signal is a 1D array (n_times,)
        smoothed_signal = moving_average(epoch_signal.flatten(), window_size)
        smoothed_signals.append(smoothed_signal)
    
    smoothed_signals = np.array(smoothed_signals)  # Shape: (n_epochs, n_times after smoothing)
    grand_averaged_signal = np.mean(smoothed_signals, axis=0)
    standard_error = np.std(smoothed_signals, axis=0) / np.sqrt(smoothed_signals.shape[0])  # SEM
    return grand_averaged_signal, standard_error


EXTRACT EEG

In [4]:
# Data Split A
train_pth_A = Path(r"data/train_A")                    # ses1 
test_pth_A_s2 = Path(r"data/test_A_ses2")              # ses2
test_pth_A_s3 = Path(r"data/test_A_ses3")              # ses3

# train data load (use .fif if exists else create new)
all_go_epochs_train_A, all_nogo_epochs_train_A = extract_eeg(folder_name=train_pth_A, fif_name=Path(r"loaded_data/train_A.fif"))
all_go_epochs_test_A_s2, all_nogo_epochs_test_A_s2 = extract_eeg(folder_name=test_pth_A_s2, fif_name=Path(r"loaded_data/test_A_s2.fif"))
all_go_epochs_test_A_s3, all_nogo_epochs_test_A_s3 = extract_eeg(folder_name=test_pth_A_s3, fif_name=Path(r"loaded_data/test_A_s3.fif"))

File loaded_data\train_A.fif already exists. Loading data.
Reading c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\loaded_data\train_A.fif ...
    Found the data of interest:
        t =   -1000.00 ...    5000.00 ms
        0 CTF compensation matrices available


c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:52: RuntimeWarning: This filename (loaded_data\train_A.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(fif_name, preload=True)


Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
File loaded_data\test_A_s2.fif already exists. Loading data.
Reading c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\loaded_data\test_A_s2.fif ...
    Found the data of interest:
        t =   -1000.00 ...    5000.00 ms
        0 CTF compensation matrices available


c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:52: RuntimeWarning: This filename (loaded_data\test_A_s2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(fif_name, preload=True)


Not setting metadata
216 matching events found
No baseline correction applied
0 projection items activated
File loaded_data\test_A_s3.fif already exists. Loading data.
Reading c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\loaded_data\test_A_s3.fif ...
    Found the data of interest:
        t =   -1000.00 ...    5000.00 ms
        0 CTF compensation matrices available


c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:52: RuntimeWarning: This filename (loaded_data\test_A_s3.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(fif_name, preload=True)


Not setting metadata
216 matching events found
No baseline correction applied
0 projection items activated


PREPROCESS

In [8]:
all_go_epochs_train_A, all_nogo_epochs_train_A = preprocess(all_go_epochs_train_A, all_nogo_epochs_train_A)
all_go_epochs_test_A_s2, all_nogo_epochs_test_A_s2 = preprocess(all_go_epochs_test_A_s2, all_nogo_epochs_test_A_s2)
all_go_epochs_test_A_s3, all_nogo_epochs_test_A_s3 = preprocess(all_go_epochs_test_A_s3, all_nogo_epochs_test_A_s3)

Setting up band-pass filter from 0.01 - 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 1.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Filter length: 168961 samples (330.002 s)



c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:238: RuntimeWarning: filter_length (168961) is longer than the signal (3073), distortion is likely. Reduce filter length or filter a longer signal.
  all_go_epochs.filter(l_freq=0.01, h_freq=1.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    6.5s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    9.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:   12.7s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:   17.1s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:   21.9s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:   27.0s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:   32.7s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:   38.8s

Setting up band-pass filter from 0.01 - 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 1.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Filter length: 168961 samples (330.002 s)



c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:239: RuntimeWarning: filter_length (168961) is longer than the signal (3073), distortion is likely. Reduce filter length or filter a longer signal.
  all_nogo_epochs.filter(l_freq=0.01, h_freq=1.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.3s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    6.8s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    9.8s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:   13.1s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:   16.8s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:   21.0s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:   26.1s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:   31.5s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:   37.

EEG channel type selected for re-referencing
EEG channel type selected for re-referencing
Projections have already been applied. Setting proj attribute to True.
Projections have already been applied. Setting proj attribute to True.
Setting up band-pass filter from 0.01 - 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 1.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Filter length: 168961 samples (330.002 s)



...

c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:242: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  all_go_epochs.set_eeg_reference('average', projection=True)  # 'average' means CAR
c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:243: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  all_nogo_epochs.set_eeg_reference('average', projection=True)
c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:238: RuntimeWarning: filter_length (168961) is longer than the signal (3073), distortion is likely. Reduce filter length or filter a longer signal.
  all_go_epochs.filter(l_freq=0.01, h_freq=1.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    

Setting up band-pass filter from 0.01 - 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 1.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Filter length: 168961 samples (330.002 s)



c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:239: RuntimeWarning: filter_length (168961) is longer than the signal (3073), distortion is likely. Reduce filter length or filter a longer signal.
  all_nogo_epochs.filter(l_freq=0.01, h_freq=1.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    3.6s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    5.7s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    8.3s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:   11.4s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:   15.0s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:   19.2s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:   23.9s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:   28.7s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:   34.

EEG channel type selected for re-referencing
EEG channel type selected for re-referencing
Projections have already been applied. Setting proj attribute to True.
Projections have already been applied. Setting proj attribute to True.


c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:242: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  all_go_epochs.set_eeg_reference('average', projection=True)  # 'average' means CAR
c:\Users\ranit\Neural_Projects\VR-BCI-Cognitive-Training\helpers.py:243: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  all_nogo_epochs.set_eeg_reference('average', projection=True)


PREPARE DATA

In [ ]:
X_train_A_rn, y_train_A_rn, X_test_A_s2_rn, y_test_A_s2_rn = prepare_data(all_go_epochs_train_A_rn, all_nogo_epochs_train_A_rn, all_go_epochs_test_A_s2_rn, all_nogo_epochs_test_A_s2_rn, 
                                                                            time_ds_factor=TIME_DOWNSAMPLE_FACTOR, use_pca=PCA, n_components=N_COMPONENTS)
_, _, X_test_A_s3_rn, y_test_A_s3_rn = prepare_data(all_go_epochs_train_A_rn, all_nogo_epochs_train_A_rn, all_go_epochs_test_A_s3_rn, all_nogo_epochs_test_A_s3_rn, 
                                                        time_ds_factor=TIME_DOWNSAMPLE_FACTOR, use_pca=PCA, n_components=N_COMPONENTS)

RUN MODEL

In [ ]:
model_A_rn, avg_accuracy_A_rn, classification_reports_A_rn, confusion_matrices_A_rn = run_model(X_train_A_rn, y_train_A_rn, mdl=MODEL, n_splits=N_FOLDS)

EVAL

In [ ]:
print("Results for Training on S1")
print(f"RN Fold Train Accuracy: {avg_accuracy_A_rn * 100:.2f}%")

print("\nResults for Test on S2")
y_pred_A_s2_rn = model_A_rn.predict(X_test_A_s2_rn)
metrics_A_s2_rn = evaluate_model(y_test_A_s2_rn, y_pred_A_s2_rn)
print(f"RN Test Accuracy: {metrics_A_s2_rn['accuracy'] * 100:.2f}%")

print("\nResults for Test on S3")
y_pred_A_s3_rn = model_A_rn.predict(X_test_A_s3_rn)
metrics_A_s3_rn = evaluate_model(y_test_A_s3_rn, y_pred_A_s3_rn)
print(f"RN Test Accuracy: {metrics_A_s3_rn['accuracy'] * 100:.2f}%")